In [1]:
# 加载文档
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_path=("https://zh.wikipedia.org/wiki/黑神话：悟空"),
)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# 文本分块
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

In [3]:
# 设置嵌入模型
from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings()

In [5]:
# 设置嵌入模型，用开源模型替换OPENAI的嵌入模型
import torch
from langchain_community.embeddings import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh",
    model_kwargs={'device': 'cuda:0' if torch.cuda.is_available() else 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

D:\Anaconda3\envs\DeepLearning\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\薛俊轩\.cache\huggingface\hub\models--BAAI--bge-small-zh. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/27.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/95.8M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# 创建向量存储
all_splits = text_splitter.split_documents(docs)

from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embed_model)
vector_store.add_documents(all_splits)

D:\Anaconda3\envs\DeepLearning\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


['7b9e745c-b63c-4730-be5c-2dbfb7d6a75c',
 'e888de8d-7e16-4517-b68b-1aeac5af242c',
 'cc50a5b0-491f-486e-8aa9-a4e2df2b10a9',
 'b1a36fa8-9f1e-4895-a43b-b1e5bea7afac',
 'e3730e72-c401-4612-a3f8-77463ae09bc4',
 '6b256a88-f408-41ef-992a-3e91c92d5ab7',
 '3eadadd3-d670-421b-b0de-cf3d00c2da67',
 '736344c1-9e8a-4533-b09f-c27c766f28a3',
 'f669fc3d-aec7-491a-8abe-8311b8761b9d',
 'bcf3e6be-5128-42d9-a8e7-03b62a02ed2d',
 '195af6e1-6847-43cb-94a8-766113c7514b',
 '83f16487-ca61-41f3-9f07-65ea4d7aca21',
 'dad0746a-35d6-4672-9fdc-7903c4e62ec6',
 '09478e4b-72a8-47ca-ab72-499d83540d15',
 'df30a0ad-37e4-437b-81ab-ab127d301e87',
 'ba040dde-d065-4581-a737-78a9e5d526e6',
 '5bef6fd0-f222-4a42-b872-90e56d91bdf8',
 'ac06d0c5-cbfd-4d04-bd4c-e3896108368a',
 '8fa3dcca-10a0-48ef-81bd-b29016e8a194',
 'b408ccc8-66e2-405c-8ef7-6315a86c366e',
 'b9caa036-9285-4ea1-8369-4a37ce15dd48',
 '4284e8ab-bd30-4f8d-9d4b-0727a26f6f24',
 '107eec90-99ad-4fe2-981e-a9a14ebaf370',
 'd78ca3bd-da24-4ae1-ae65-3b3049e56199',
 'dd3a6aa8-a2a6-

In [9]:
# 构建用户查询的问题
question = "黑神话：悟空的游戏类型是什么？"

In [10]:
# 在向量存储中搜索相关文档，并准备上下文内容
retrieved_docs = vector_store.similarity_search(
    question,
    k=3
)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)


In [11]:
# 构建提示词模板
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """基于以下上下文回答问题。如果上下文没有相关信息，请说
    "我无法从提供的上下文中找到相关内容"。
    上下文：{context}
    问题：{question}
    回答："""
)

In [17]:
# 使用大模型生成回答
from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.1,
    max_tokens=512,
    api_key="你的key"
)
answer = llm.invoke(
    prompt.format(
        question=question,
        context=docs_content
    )
)
print("\n 最终答案：",answer.content)


 最终答案： 根据提供的上下文，《黑神话：悟空》的游戏类型是**动作角色扮演游戏**（Action Role-Playing Game, ARPG）。
